### Partition face dataset into 8:2 training:test

### Apply PCA on training data by use of eigenvectors and eigenvalues of covaraiance matrix S= (1/N)A(A^T)

### Apply PCA using eigenvectors and eigenvalues of (1/N)(A^T)A

In [240]:
# PCA source: Week 2 slides - Manifold Learning

import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

np.random.seed(42)

data = loadmat('face.mat')

X = data['X'] # flat images dim (2576,520)
# l = data['l'].flatten()  # image Labels (1,520)

# Does a 8/2 split of dataset (8 images / 2 images of each person). Training data shuffled. 
def Q1_partition(data):
    X = data['X'] # flat images dim (2576,520)
    X_train, X_test = [], []
    for i in range(0, 520, 10):
        im_train, im_test = train_test_split(X[:, i: i + 10].T, test_size=0.2, random_state=42)
        X_train.append(im_train)    
        X_test.append(im_test)  
  
    return  np.array(X_train).reshape(-1, 2576), np.array(X_test).reshape(-1, 2576)

In [241]:
# Splits data into five equal sets of 2 images per person x 52 people = 104 
def Q2_partition(data, num_partitions):
    X = data['X'] # flat images dim (2576,520)
    t = [[] for _ in range(num_partitions)]
    for i in range(0, 520, 10):
        slice = X[:, i: i + 10].T
        for j in range(num_partitions):
            t[j].append(slice[j*2:(j+1)*2])
       
    for i in range(num_partitions-1):
        t[i] = np.array(t[i]).reshape(-1, 2576) # Also shuffle 
        np.random.shuffle(t[i])
    t[-1] = np.array(t[4]).reshape(-1, 2576)

    return t


In [282]:
def display_image(image, label):
    # X[:, image_index]
    image_height, image_width = 46, 56 
    image = image.reshape((image_height, image_width)).T 
    # label = l[image_index]

    plt.imshow(image, cmap='gray', aspect='auto')
    plt.title(f"Face Image - Label: {label}")
    plt.axis('off')
    plt.show()
    # display_image(X_train[1:2].T, "hey")
    # display_image(t1[i:i+1].T, "Hey")

def get_sorted_eigen(M):
    eigenvalues, eigenvectors = np.linalg.eig(M)
    eigenvalues, eigenvectors = np.real(eigenvalues), np.real(eigenvectors)

    sorted_indices = np.argsort(eigenvalues)[::-1]

    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    return sorted_eigenvalues, sorted_eigenvectors

# S = Covariance Matrix, A = mean centred data, N = #samples
def PCA(batch, threshold = 0.95):
    _, _, S = process_batch(batch)
    eigenvalues, eigenvectors = get_sorted_eigen(S)
    k = choose_principal_components(threshold, eigenvalues)
    return eigenvalues[:k + 1], eigenvectors[:, :k + 1]

def choose_principal_components(threshold, eigenvalues):
    covariance_ratios = eigenvalues/np.sum(eigenvalues)
    cum_var = 0
    for k, ratio in enumerate(covariance_ratios):
        cum_var += ratio
        if cum_var >= threshold: return k 

def process_batch(batch):
    mean = np.mean(batch, axis=0)
    A = batch - mean
    print(f'batch: {batch.shape}')
    print(f'A: {A.shape}')
    N = A.shape[1]
    S = (1/N) * np.dot(A.T, A)
    print(f'S: {S.shape}')
    print(f'Mean: {mean.shape}')
    return N, mean, S

In [283]:
# They cannot possibly be the same since they have different dimensions. 
X_train, X_test = Q1_partition(data)
##### Look here FELIX - AAT: S = (1/N) A * A^T, ATA: S = (1/N) A^T * A  
eigenvalues_AAT, eigenvectors_AAT = PCA(X_train)
eigenvalues_ATA, eigenvectors_ATA = PCA(X_train.T)
##########

batch: (416, 2576)
A: (416, 2576)
S: (2576, 2576)
Mean: (2576,)
batch: (2576, 416)
A: (2576, 416)
S: (416, 416)
Mean: (416,)


In [284]:
t = Q2_partition(data,5)


batch = t[0]
N, mean, S = process_batch(X_train)


batch: (416, 2576)
A: (416, 2576)
S: (2576, 2576)
Mean: (2576,)


In [285]:
def incremental_PCA(data, num_partitions = 5):
    t = Q2_partition(data, num_partitions)
    
    N1, mu1, S1 = process_batch(t[0])

    for batch in t[1:-1]:  # every batch except first (already initialized) and last (test)
        N2, mu2, S2 = process_batch(batch)
        print(batch.shape)

        N3 = N1 + N2
        mu3 = (N1*mu1 + N2*mu2)/N3
        mu12 = mu1-mu2
        term1 = (N1/N3) * S1
        term2 = (N2/N3)*S2
        term3 = (N1*N2)/(N3**2) * np.outer(mu12, mu12.T)  
        S3 =  term1 + term2 + term3 

        N1, mu1, S1 = N3, mu3, S3

In [286]:
incremental_PCA(data)

batch: (104, 2576)
A: (104, 2576)
S: (2576, 2576)
Mean: (2576,)
batch: (104, 2576)
A: (104, 2576)
S: (2576, 2576)
Mean: (2576,)
(104, 2576)
batch: (104, 2576)
A: (104, 2576)
S: (2576, 2576)
Mean: (2576,)
(104, 2576)
batch: (104, 2576)
A: (104, 2576)
S: (2576, 2576)
Mean: (2576,)
(104, 2576)
